In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [2]:
import pandas as pd

df = pd.read_csv("movies_cleaned.csv")

In [3]:
df

,Title,Overview,Popularity,Vote_Count,Vote_Average,Original_Language,Genre,Poster_Url,Release_Year,text_for_embedding
0,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,en,"action, adventure, science fiction",https://image.tmdb.org/t/p/original/1g0dhYtq4i...,2021,Title: Spider-Man: No Way Home | Genres: actio...
1,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,en,"crime, mystery, thriller",https://image.tmdb.org/t/p/original/74xTEgt7R3...,2022,"Title: The Batman | Genres: crime, mystery, th..."
2,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,en,thriller,https://image.tmdb.org/t/p/original/vDHsLnOWKl...,2022,Title: No Exit | Genres: thriller | Overview: ...
3,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,en,"animation, comedy, family, fantasy",https://image.tmdb.org/t/p/original/4j0PNHkMr5...,2021,"Title: Encanto | Genres: animation, comedy, fa..."
4,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,en,"action, adventure, thriller, war",https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...,2021,"Title: The King's Man | Genres: action, advent..."
...,...,...,...,...,...,...,...,...,...,...
9802,Badlands,A dramatization of the Starkweather-Fugate kil...,13.357,896,7.6,en,"drama, crime",https://image.tmdb.org/t/p/original/z81rBzHNgi...,1973,"Title: Badlands | Genres: drama, crime | Overv..."
9803,Violent Delights,A female vampire falls in love with a man she ...,13.356,8,3.5,es,horror,https://image.tmdb.org/t/p/original/4b6HY7rud6...,2020,Title: Violent Delights | Genres: horror | Ove...
9804,The Offering,When young and successful reporter Jamie finds...,13.355,94,5.0,en,"mystery, thriller, horror",https://image.tmdb.org/t/p/original/h4uMM1wOhz...,2016,"Title: The Offering | Genres: mystery, thrille..."
9805,The United States vs. Billie Holiday,Billie Holiday spent much of her career being ...,13.354,152,6.7,en,"music, drama, history",https://image.tmdb.org/t/p/original/vEzkxuE2sJ...,2021,Title: The United States vs. Billie Holiday | ...


### jadiin file txt, karena textsplitter hanya meminta file .txt.

In [4]:
df["text_for_embedding"].to_csv("text_for_embedding.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

### Pake chunk_size seminimal mungkin, karena kalo synopsis lebih panjang dari 1, maka akan keambil semua full text.

In [10]:
loader = TextLoader("text_for_embedding.txt", encoding="utf-8") 
raw_documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 346, which is longer than the specified 1
Created a chunk of size 227, which is longer than the specified 1
Created a chunk of size 305, which is longer than the specified 1
Created a chunk of size 592, which is longer than the specified 1
Created a chunk of size 232, which is longer than the specified 1
Created a chunk of size 545, which is longer than the specified 1
Created a chunk of size 284, which is longer than the specified 1
Created a chunk of size 265, which is longer than the specified 1
Created a chunk of size 206, which is longer than the specified 1
Created a chunk of size 297, which is longer than the specified 1
Created a chunk of size 390, which is longer than the specified 1
Created a chunk of size 620, which is longer than the specified 1
Created a chunk of size 214, which is longer than the specified 1
Created a chunk of size 235, which is longer than the specified 1
Created a chunk of size 265, which is longer than the specified 1
Created a 

In [11]:
documents[0]

Document(metadata={'source': 'text_for_embedding.txt'}, page_content='Title: Spider-Man: No Way Home | Genres: action, adventure, science fiction | Overview: Peter Parker is unmasked and no longer able to separate his normal life from the high-stakes of being a super-hero. When he asks for help from Doctor Strange the stakes become even more dangerous, forcing him to discover what it truly means to be Spider-Man.')

### ambil model embedding dari huggingface, disini diubah dari chroma dataset (vector database), karena embedding mengubah kata jadi vector.

In [13]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Jonathan\miniconda3\envs\free_download\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jonathan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
movies_db = Chroma.from_documents(documents, embedding=embeddings)

In [26]:
def retrieve_movie_recommendations(query: str, top_k: int = 50) -> pd.DataFrame:
    recs = movies_db.similarity_search(query, k=top_k)

    title_list = []
    for i in range(len(recs)):
        raw_title = recs[i].page_content.split("|")[0].strip()
        clean_title = raw_title.replace("Title:", "").strip()
        title_list.append(clean_title)

    return df[df["Title"].isin(title_list)]

In [28]:
retrieve_movie_recommendations("spiderman")

,Title,Overview,Popularity,Vote_Count,Vote_Average,Original_Language,Genre,Poster_Url,Release_Year,text_for_embedding
0,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,en,"action, adventure, science fiction",https://image.tmdb.org/t/p/original/1g0dhYtq4i...,2021,Title: Spider-Man: No Way Home | Genres: actio...
90,The Amazing Spider-Man,Peter Parker is an outcast high schooler aband...,306.376,14564,6.7,en,"action, adventure, fantasy",https://image.tmdb.org/t/p/original/fSbqPbqXa7...,2012,Title: The Amazing Spider-Man | Genres: action...
132,The Amazing Spider-Man 2,"For Peter Parker, life is busy. Between taking...",231.441,10763,6.5,en,"action, adventure, fantasy",https://image.tmdb.org/t/p/original/c3e9e18SSl...,2014,Title: The Amazing Spider-Man 2 | Genres: acti...
144,Spider-Man: Far From Home,Peter Parker and his friends go on a summer tr...,224.034,12176,7.5,en,"action, adventure, science fiction",https://image.tmdb.org/t/p/original/4q2NNj4S5d...,2019,Title: Spider-Man: Far From Home | Genres: act...
168,Spider-Man: Homecoming,Following the events of Captain America: Civil...,207.817,18174,7.4,en,"action, adventure, science fiction, drama",https://image.tmdb.org/t/p/original/c24sv2weTH...,2017,Title: Spider-Man: Homecoming | Genres: action...
170,Spider-Man,After being bitten by a genetically altered sp...,206.376,15336,7.2,en,"fantasy, action",https://image.tmdb.org/t/p/original/gh4cZbhZxy...,2002,"Title: Spider-Man | Genres: fantasy, action | ..."
201,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,186.125,11340,6.3,en,"fantasy, action, adventure",https://image.tmdb.org/t/p/original/qFmwhVUoUS...,2007,"Title: Spider-Man 3 | Genres: fantasy, action,..."
916,Spider-Man: Across the Spider-Verse (Part One),Miles Morales returns for the next chapter of ...,69.174,0,0.0,en,"animation, action, science fiction, comedy, ad...",https://image.tmdb.org/t/p/original/l2hjrByNNo...,2022,Title: Spider-Man: Across the Spider-Verse (Pa...
1077,Megamind,Bumbling supervillain Megamind finally defeats...,62.303,5497,6.9,en,"animation, action, comedy, family, science fic...",https://image.tmdb.org/t/p/original/7l5OY9oeJy...,2010,"Title: Megamind | Genres: animation, action, c..."
1361,Spider-Man 2,Peter Parker is going through a major identity...,53.750,12315,7.2,en,"action, adventure, fantasy",https://image.tmdb.org/t/p/original/olxpyq9kJA...,2004,"Title: Spider-Man 2 | Genres: action, adventur..."
